In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from math import sqrt
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import skew

In [15]:
df = pd.read_csv("analysis_data.csv")

In [13]:
df

,id,targeting_score,visual_appeal,contextual_relevance,headline_length,cta_strength,position_on_page,ad_format,age_group,gender,...,headline_word_count,headline_power_words,body_text_length,body_word_count,body_sentiment,headline_question,headline_numbers,body_keyword_density,body_readability_score,CTR
0,4316,4.0,1.118593,0.0,73.0,0.0,In-Content,Text,25-34,Male,...,12.0,1.0,105.0,22.0,-1.668241,1.0,1.0,0.075298,61.483619,0.011966
1,7271,3.0,4.890355,0.0,93.0,3.0,Side Banner,Text,25-34,Male,...,14.0,1.0,194.0,20.0,3.347280,1.0,0.0,0.097989,54.775205,0.000000
2,6477,3.0,4.744698,1.0,96.0,3.0,In-Content,Text,35-44,Female,...,15.0,1.0,164.0,38.0,-1.015219,1.0,0.0,0.018412,66.367516,0.000000
3,4847,6.0,4.838057,0.0,93.0,5.0,Side Banner,Text,18-24,Female,...,6.0,0.0,61.0,37.0,0.066430,1.0,0.0,0.068744,66.084536,0.000681
4,4566,3.0,1.358385,0.0,79.0,3.0,Side Banner,Text,18-24,Female,...,12.0,0.0,191.0,1.0,-0.996762,1.0,0.0,0.043858,89.647942,0.005134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2277,15.0,4.028834,1.0,27.0,3.0,In-Content,Text,18-24,Male,...,12.0,0.0,136.0,38.0,-2.303541,0.0,0.0,0.052186,66.904201,0.794704
3996,1400,7.0,14.913071,0.0,20.0,3.0,Top Banner,Image,25-34,Female,...,2.0,1.0,44.0,8.0,-0.897440,1.0,0.0,0.074194,64.402474,0.964406
3997,4225,0.0,15.506949,0.0,21.0,7.0,Top Banner,Image,35-44,Male,...,3.0,1.0,48.0,8.0,-1.489064,1.0,0.0,0.041043,83.293061,0.895757
3998,4974,20.0,1.416915,0.0,10.0,2.0,Side Banner,Video,25-34,Female,...,4.0,0.0,1.0,9.0,-1.430613,0.0,0.0,0.098981,82.190934,1.645827


In [17]:
from sklearn.impute import SimpleImputer
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

In [18]:
imputer_median = SimpleImputer(strategy='median')
df[numerical_cols] = imputer_median.fit_transform(df[numerical_cols])

In [19]:
imputer_most = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer_most.fit_transform(df[categorical_cols])

In [20]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['age_group']= label_encoder.fit_transform(df['age_group']) 
df['gender']= label_encoder.fit_transform(df['gender']) 
df['location']= label_encoder.fit_transform(df['location']) 
df['device_type']= label_encoder.fit_transform(df['device_type']) 
df['day_of_week']= label_encoder.fit_transform(df['day_of_week']) 
df['ad_format']= label_encoder.fit_transform(df['ad_format']) 
df['time_of_day']= label_encoder.fit_transform(df['time_of_day']) 
df['position_on_page']= label_encoder.fit_transform(df['position_on_page']) 

In [21]:
df['is_zero'] = (df['CTR'] != 0).astype(int)

In [22]:
df

,id,targeting_score,visual_appeal,contextual_relevance,headline_length,cta_strength,position_on_page,ad_format,age_group,gender,...,headline_power_words,body_text_length,body_word_count,body_sentiment,headline_question,headline_numbers,body_keyword_density,body_readability_score,CTR,is_zero
0,4316.0,4.0,1.118593,0.0,73.0,0.0,0,1,1,1,...,1.0,105.0,22.0,-1.668241,1.0,1.0,0.075298,61.483619,0.011966,1
1,7271.0,3.0,4.890355,0.0,93.0,3.0,1,1,1,1,...,1.0,194.0,20.0,3.347280,1.0,0.0,0.097989,54.775205,0.000000,0
2,6477.0,3.0,4.744698,1.0,96.0,3.0,0,1,2,0,...,1.0,164.0,38.0,-1.015219,1.0,0.0,0.018412,66.367516,0.000000,0
3,4847.0,6.0,4.838057,0.0,93.0,5.0,1,1,0,0,...,0.0,61.0,37.0,0.066430,1.0,0.0,0.068744,66.084536,0.000681,1
4,4566.0,3.0,1.358385,0.0,79.0,3.0,1,1,0,0,...,0.0,191.0,1.0,-0.996762,1.0,0.0,0.043858,89.647942,0.005134,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2277.0,15.0,4.028834,1.0,27.0,3.0,0,1,0,1,...,0.0,136.0,38.0,-2.303541,0.0,0.0,0.052186,66.904201,0.794704,1
3996,1400.0,7.0,14.913071,0.0,20.0,3.0,2,0,1,0,...,1.0,44.0,8.0,-0.897440,1.0,0.0,0.074194,64.402474,0.964406,1
3997,4225.0,0.0,15.506949,0.0,21.0,7.0,2,0,2,1,...,1.0,48.0,8.0,-1.489064,1.0,0.0,0.041043,83.293061,0.895757,1
3998,4974.0,20.0,1.416915,0.0,10.0,2.0,1,2,1,0,...,0.0,1.0,9.0,-1.430613,0.0,0.0,0.098981,82.190934,1.645827,1


In [23]:
df.corr()['CTR'].sort_values(ascending = False)

CTR                       1.000000
visual_appeal             0.501161
targeting_score           0.333659
is_zero                   0.270571
cta_strength              0.116402
ad_format                 0.097740
contextual_relevance      0.041791
position_on_page          0.030512
device_type               0.029685
body_sentiment            0.012195
day_of_week               0.008972
body_readability_score    0.007647
seasonality               0.002309
body_keyword_density      0.002135
ad_frequency              0.000978
gender                    0.000167
time_of_day              -0.002033
brand_familiarity        -0.002719
headline_power_words     -0.008420
headline_numbers         -0.008534
headline_question        -0.014959
market_saturation        -0.017850
id                       -0.018740
age_group                -0.020310
location                 -0.027622
headline_sentiment       -0.042016
body_text_length         -0.056006
headline_word_count      -0.059855
body_word_count     

## Skewness Check

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew 
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
sns.distplot(df['CTR'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="CTR")
ax.set(title="CTR distribution")
sns.despine(trim=True, left=True)
plt.show()
print("skew value: ", skew(df['CTR']))

In [ ]:
sns.histplot(data=df, x='CTR', bins=15, kde=True)
plt.show()

## Log Transformation

In [ ]:
df.skew().sort_values(ascending = False)

In [7]:
#df['CTR'] = np.log(df['CTR']+ 1)
#df['brand_familiarity'], lambda_value = boxcox(df['brand_familiarity'])
#df['market_saturation'], lambda_value = boxcox(df['market_saturation'])
#df['targeting_score'], lambda_value = boxcox(df['targeting_score'])
#df['cta_strength'], lambda_value = boxcox(df['cta_strength'])
#df['body_text_length'], lambda_value = boxcox(df['body_text_length'])
#df['body_word_count'], lambda_value = boxcox(df['body_word_count'])
#df['age_group'], lambda_value = boxcox(df['age_group'])

df['brand_familiarity'] = np.log(df['brand_familiarity']+ 1)
df['market_saturation'] = np.log(df['market_saturation']+ 1)
df['targeting_score'] = np.log(df['targeting_score']+ 1)
df['cta_strength'] = np.log(df['cta_strength']+ 1)
df['body_text_length'] = np.log(df['body_text_length']+ 1)
df['body_word_count'] = np.log(df['body_word_count']+ 1)
df['headline_length'] = np.log(df['headline_length']+ 1)
df['headline_word_count'] = np.log(df['headline_word_count']+ 1)
df['time_of_day'] = np.log(df['time_of_day']+ 1)
df['age_group'] = np.log(df['age_group']+ 1) #worked

In [ ]:
from scipy import stats
from scipy.stats import boxcox

#df['CTR'] = np.log(df['CTR']+ 1)
#df['is_zero'] = df['is_zero'] ** 2
#new_dataset['body_text_length'] = new_dataset['body_text_length'] ** 2
#new_dataset['visual_appeal'] = new_dataset['visual_appeal'] ** 2

In [ ]:
df['CTR'] = np.log(df['CTR']+ 1)

In [ ]:
df['CTR'] = np.log(df['CTR']+ 1)

In [ ]:
df.skew().sort_values(ascending = False)

## Outlier Detection and Imputation

In [ ]:
ctr = np.abs(stats.zscore(df['CTR'])) #done

In [8]:
from scipy import stats
#ctr = np.abs(stats.zscore(df['CTR'])) #done
hl = np.abs(stats.zscore(df['headline_length']))
va = np.abs(stats.zscore(df['visual_appeal'])) #done
ge = np.abs(stats.zscore(df['gender']))
ag = np.abs(stats.zscore(df['age_group']))
lo = np.abs(stats.zscore(df['location']))
af = np.abs(stats.zscore(df['ad_format'])) #done
hs = np.abs(stats.zscore(df['headline_sentiment']))
brs = np.abs(stats.zscore(df['body_readability_score']))
bs = np.abs(stats.zscore(df['body_sentiment']))
se = np.abs(stats.zscore(df['seasonality']))
bkd = np.abs(stats.zscore(df['body_keyword_density']))
dow = np.abs(stats.zscore(df['day_of_week']))
bf = np.abs(stats.zscore(df['brand_familiarity']))
cs = np.abs(stats.zscore(df['cta_strength'])) #done
ms = np.abs(stats.zscore(df['market_saturation']))
ts = np.abs(stats.zscore(df['targeting_score'])) #done
dt = np.abs(stats.zscore(df['device_type']))
bwc = np.abs(stats.zscore(df['body_word_count']))
btl = np.abs(stats.zscore(df['body_text_length']))

In [ ]:
#zero = np.abs(stats.zscore(df['is_zero']))

In [10]:
threshold_z = 3

#outlier_indices_zero = np.where((zero > threshold_z) | (zero < -threshold_z))[0] #yes
outlier_indices_va = np.where((va > threshold_z) | (va < -threshold_z))[0] #yes
#outlier_indices_ctr = np.where((ctr > threshold_z) | (ctr < -threshold_z))[0]  #yes
outlier_indices_ts = np.where((ts > threshold_z) | (ts < -threshold_z))[0] #yes
outlier_indices_cs = np.where((cs > threshold_z) | (cs < -threshold_z))[0] #none
outlier_indices_af = np.where((af > threshold_z) | (af < -threshold_z))[0] #none
#outlier_indices_cr = np.where((cr > threshold_z) | (cr < -threshold_z))[0] #none
outlier_indices_hl = np.where((hl > threshold_z) | (hl < -threshold_z))[0] #none
outlier_indices_ge = np.where((ge > threshold_z) | (ge < -threshold_z))[0] #none
outlier_indices_ag = np.where((ag > threshold_z) | (ag < -threshold_z))[0] #none
outlier_indices_lo = np.where((lo > threshold_z) | (lo < -threshold_z))[0] #none
outlier_indices_af = np.where((af > threshold_z) | (af < -threshold_z))[0] #none
outlier_indices_hs = np.where((hs > threshold_z) | (hs < -threshold_z))[0] #yes
outlier_indices_brs = np.where((brs > threshold_z) | (brs < -threshold_z))[0] #none
outlier_indices_bs = np.where((bs > threshold_z) | (bs < -threshold_z))[0] #yes
outlier_indices_se = np.where((se > threshold_z) | (se < -threshold_z))[0] #none
outlier_indices_bkd = np.where((bkd > threshold_z) | (bkd < -threshold_z))[0] #none
outlier_indices_dow = np.where((dow > threshold_z) | (dow < -threshold_z))[0] #none
outlier_indices_bf = np.where((bf > threshold_z) | (bf < -threshold_z))[0] #yes
outlier_indices_cs = np.where((cs > threshold_z) | (cs < -threshold_z))[0] #none
outlier_indices_ms = np.where((ms > threshold_z) | (ms < -threshold_z))[0] #none
outlier_indices_ts = np.where((ts > threshold_z) | (ts < -threshold_z))[0] #yes
outlier_indices_dt = np.where((dt > threshold_z) | (dt < -threshold_z))[0] #none
outlier_indices_bwc = np.where((bwc > threshold_z) | (bwc < -threshold_z))[0] #none
outlier_indices_btl = np.where((btl > threshold_z) | (btl < -threshold_z))[0] #yes

In [ ]:
outlier_indices_va

In [ ]:
outlier_indices_ctr

In [ ]:
outlier_indices_hs

In [ ]:
outlier_indices_bs

In [ ]:
outlier_indices_bf 

In [ ]:
outlier_indices_ts

In [ ]:
outlier_indices_btl

In [ ]:
median_ctr = df['CTR'].median()

In [11]:
median_va = df['visual_appeal'].median()
median_ts = df['targeting_score'].median()
#median_ctr = df['CTR'].median()
median_hs = df['headline_sentiment'].median()
median_bs = df['body_sentiment'].median()
median_bf = df['brand_familiarity'].median()
median_btl = df['body_text_length'].median()

In [12]:
df.loc[outlier_indices_va, 'visual_appeal'] = median_va
df.loc[outlier_indices_ts, 'targeting_score'] = median_ts
#df.loc[outlier_indices_ctr, 'CTR'] = median_ctr
df.loc[outlier_indices_hs, 'headline_sentiment'] = median_hs
df.loc[outlier_indices_bs, 'body_sentiment'] = median_bs
df.loc[outlier_indices_bf, 'brand_familiarity'] = median_bf
df.loc[outlier_indices_btl, 'body_text_length'] = median_btl

In [ ]:
df.skew().sort_values(ascending = False)

In [9]:
df.corr()['CTR'].sort_values(ascending = False)

CTR                       1.000000
visual_appeal             0.501161
targeting_score           0.333659
cta_strength              0.116402
ad_format                 0.097740
contextual_relevance      0.041791
position_on_page          0.030512
device_type               0.029685
body_sentiment            0.012195
day_of_week               0.008972
body_readability_score    0.007647
seasonality               0.002309
body_keyword_density      0.002135
ad_frequency              0.000978
gender                    0.000167
time_of_day              -0.002033
brand_familiarity        -0.002719
headline_power_words     -0.008420
headline_numbers         -0.008534
headline_question        -0.014959
market_saturation        -0.017850
id                       -0.018740
age_group                -0.020310
location                 -0.027622
headline_sentiment       -0.042016
body_text_length         -0.056006
headline_word_count      -0.059855
body_word_count          -0.064256
headline_length     

In [13]:
train_df = df[["visual_appeal", "targeting_score", "cta_strength", "ad_format", "contextual_relevance", "CTR"]]

In [ ]:
train_df.isna().sum()

## Training

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge

In [15]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
minmax = MinMaxScaler()
df_normalized = minmax.fit_transform(train_df)
df_normalized = pd.DataFrame(df_normalized, columns=train_df.columns)

In [19]:
df_normalized

,visual_appeal,targeting_score,cta_strength,ad_format,contextual_relevance,CTR
0,0.367940,0.528634,0.000000,0.5,0.0,0.003195
1,0.527365,0.455340,0.470818,0.5,0.0,0.000000
2,0.521208,0.455340,0.470818,0.5,1.0,0.000000
3,0.525154,0.639151,0.608523,0.5,0.0,0.000182
4,0.378076,0.455340,0.470818,0.5,0.0,0.001371
...,...,...,...,...,...,...
3995,0.490950,0.910681,0.470818,0.5,1.0,0.212202
3996,0.951004,0.683011,0.470818,0.0,0.0,0.257516
3997,0.976106,0.000000,0.706227,0.0,0.0,0.239186
3998,0.380550,1.000000,0.373114,1.0,0.0,0.439470


In [20]:
pipeline = Pipeline(steps=[
    ('model', GradientBoostingRegressor())  
  #  ('model', Ridge())
])

y = df_normalized['CTR']
X = df_normalized.drop('CTR', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, shuffle = True, random_state = 42)

In [21]:
cv = KFold(n_splits=15, shuffle=True, random_state=1)

In [22]:
grid = {
    'model__n_estimators': [100, 200, 300],  
    'model__learning_rate': [0.01, 0.1, 0.2],  
    'model__max_depth': [3, 5, 7] 
}

In [23]:
search = GridSearchCV(pipeline, grid, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
results = search.fit(X_train, y_train)

In [24]:
best_model = results.best_estimator_

In [49]:
print("Best Parameters:", results.best_params_)

Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 200}


In [46]:
y_pred = best_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 0.03710097789594266


In [ ]:
y_pred

# Testing

In [26]:
test_df = pd.read_csv("scoring_data.csv")

In [27]:
imputer = SimpleImputer(strategy='mean') 

numerical_cols_test = test_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols_test = test_df.select_dtypes(include=['object']).columns

In [28]:
imputer_median = SimpleImputer(strategy='median')
test_df[numerical_cols_test] = imputer_median.fit_transform(test_df[numerical_cols_test])

imputer_most = SimpleImputer(strategy='most_frequent')
test_df[categorical_cols_test] = imputer_most.fit_transform(test_df[categorical_cols_test])

In [29]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

test_df['age_group']= label_encoder.fit_transform(test_df['age_group']) 
test_df['gender']= label_encoder.fit_transform(test_df['gender']) 
test_df['location']= label_encoder.fit_transform(test_df['location']) 
test_df['device_type']= label_encoder.fit_transform(test_df['device_type']) 
test_df['day_of_week']= label_encoder.fit_transform(test_df['day_of_week']) 
test_df['ad_format']= label_encoder.fit_transform(test_df['ad_format']) 
test_df['time_of_day']= label_encoder.fit_transform(test_df['time_of_day']) 
test_df['position_on_page']= label_encoder.fit_transform(test_df['position_on_page']) 

## Skewness Check

In [ ]:
test_df['is_zero']= df['is_zero']

In [45]:
test_df.skew().sort_values(ascending = False)

CTR                       2.712644
headline_length           0.898307
headline_word_count       0.723191
time_of_day               0.378143
visual_appeal             0.242444
gender                    0.213702
seasonality               0.108320
day_of_week               0.083362
body_readability_score    0.022936
headline_sentiment        0.020468
body_keyword_density      0.005142
position_on_page          0.003744
body_sentiment           -0.011812
ad_frequency             -0.024512
ad_format                -0.029196
id                       -0.047338
body_text_length         -0.060726
market_saturation        -0.083123
age_group                -0.095668
headline_power_words     -0.096255
headline_numbers         -0.132487
targeting_score          -0.139761
headline_question        -0.152670
cta_strength             -0.162951
contextual_relevance     -0.164801
location                 -0.166878
brand_familiarity        -0.387485
device_type              -0.554649
body_word_count     

In [30]:
test_df['targeting_score'] = np.log(test_df['targeting_score']+1)
test_df['age_group'] = np.log(test_df['age_group']+ 1)
test_df['market_saturation'] = np.log(test_df['market_saturation']+ 1)
test_df['cta_strength'] = np.log(test_df['cta_strength']+ 1)
test_df['gender'] = np.log(test_df['gender']+ 1)
test_df['brand_familiarity'] = np.log(test_df['brand_familiarity']+ 1)
test_df['body_text_length'] = np.log(test_df['body_text_length']+ 1)
test_df['body_word_count'] = np.log(test_df['body_word_count']+ 1)

In [31]:
test_df['body_text_length'] = test_df['body_text_length'] ** 2
#test_df['is_zero'] = test_df['is_zero'] ** 2

## OD and Imputation

In [33]:
hl_test = np.abs(stats.zscore(test_df['headline_length'])) 
hwc_test = np.abs(stats.zscore(test_df['headline_word_count'])) 
tod_test = np.abs(stats.zscore(test_df['time_of_day'])) 
pop_test = np.abs(stats.zscore(test_df['position_on_page'])) 
hpw_test = np.abs(stats.zscore(test_df['headline_power_words'])) 
ad_frequency_test = np.abs(stats.zscore(test_df['ad_frequency']))
hn_test = np.abs(stats.zscore(test_df['headline_numbers']))  
cr_test = np.abs(stats.zscore(test_df['contextual_relevance'])) 
hq_test = np.abs(stats.zscore(test_df['headline_question'])) 
va_test = np.abs(stats.zscore(test_df['visual_appeal'])) 
ge_test = np.abs(stats.zscore(test_df['gender'])) 
ag_test = np.abs(stats.zscore(test_df['age_group'])) 
lo_test = np.abs(stats.zscore(test_df['location'])) 
af_test = np.abs(stats.zscore(test_df['ad_format'])) 
hs_test = np.abs(stats.zscore(test_df['headline_sentiment'])) 
brs_test = np.abs(stats.zscore(test_df['body_readability_score'])) 
bs_test = np.abs(stats.zscore(test_df['body_sentiment'])) 
se_test = np.abs(stats.zscore(test_df['seasonality'])) 
bkd_test = np.abs(stats.zscore(test_df['body_keyword_density'])) 
dow_test = np.abs(stats.zscore(test_df['day_of_week'])) 
bf_test = np.abs(stats.zscore(test_df['brand_familiarity'])) 
cs_test = np.abs(stats.zscore(test_df['cta_strength'])) 
ms_test = np.abs(stats.zscore(test_df['market_saturation']))
ts_test = np.abs(stats.zscore(test_df['targeting_score'])) 
dt_test = np.abs(stats.zscore(test_df['device_type'])) #done
bwc_test = np.abs(stats.zscore(test_df['body_word_count']))
btl_test = np.abs(stats.zscore(test_df['body_text_length'])) #done

In [34]:
outlier_indices_hl_test = np.where((hl_test > threshold_z) | (hl_test < -threshold_z))[0]  #none
outlier_indices_tod_testt = np.where((tod_test > threshold_z) | (tod_test < -threshold_z))[0]  #none
outlier_indices_hwc_test = np.where((hwc_test > threshold_z) | (hwc_test < -threshold_z))[0]  #none
outlier_indices_pop_test = np.where((pop_test > threshold_z) | (pop_test < -threshold_z))[0]  #none
outlier_indices_hpw_test = np.where((hpw_test > threshold_z) | (hpw_test < -threshold_z))[0] #none
outlier_indices_ad_frequency_test = np.where((ad_frequency_test > threshold_z) | (ad_frequency_test < -threshold_z))[0] #none
outlier_indices_hn_test = np.where((hn_test > threshold_z) | (hn_test < -threshold_z))[0]  #none
outlier_indices_hq_test = np.where((hq_test > threshold_z) | (hq_test < -threshold_z))[0] #none
outlier_indices_ge_test = np.where((ge_test > threshold_z) | (ge_test < -threshold_z))[0] #none
outlier_indices_ag_test = np.where((ag_test > threshold_z) | (ag_test < -threshold_z))[0] #none
outlier_indices_lo_test = np.where((lo_test > threshold_z) | (lo_test < -threshold_z))[0] #none
outlier_indices_hs_test = np.where((hs_test > threshold_z) | (hs_test < -threshold_z))[0] 
outlier_indices_brs_test = np.where((brs_test > threshold_z) | (brs_test < -threshold_z))[0] #none
outlier_indices_bs_test = np.where((bs_test > threshold_z) | (bs_test < -threshold_z))[0] 
outlier_indices_se_test = np.where((se_test > threshold_z) | (se_test < -threshold_z))[0] #none

outlier_indices_bkd_test = np.where((bkd_test > threshold_z) | (bkd_test < -threshold_z))[0]  #none
outlier_indices_dow_test = np.where((dow_test > threshold_z) | (dow_test < -threshold_z))[0] #none
outlier_indices_bf_test = np.where((bf_test > threshold_z) | (bf_test < -threshold_z))[0] #none
outlier_indices_ms_test = np.where((ms_test > threshold_z) | (ms_test < -threshold_z))[0] #none
outlier_indices_dt_test = np.where((dt_test > threshold_z) | (dt_test < -threshold_z))[0] #none
outlier_indices_btl_test = np.where((btl_test > threshold_z) | (btl_test < -threshold_z))[0]  #none

outlier_indices_va_test = np.where((va_test > threshold_z) | (va_test < -threshold_z))[0]
outlier_indices_ts_test= np.where((ts_test > threshold_z) | (ts_test < -threshold_z))[0] #none
outlier_indices_cs_test = np.where((cs_test > threshold_z) | (cs_test < -threshold_z))[0] #none
outlier_indices_af_test = np.where((af > threshold_z) | (af < -threshold_z))[0] #none
outlier_indices_cr_test = np.where((cr_test > threshold_z) | (cr_test < -threshold_z))[0] #none

In [ ]:
outlier_indices_hs_test 

In [ ]:
outlier_indices_bs_test

In [ ]:
outlier_indices_va_test 

In [35]:
median_hs_test = test_df['headline_sentiment'].median()
test_df.loc[outlier_indices_hs_test, 'headline_sentiment'] = median_hs_test

median_bs_test = test_df['body_sentiment'].median()
test_df.loc[outlier_indices_bs_test, 'body_sentiment'] = median_bs_test

median_va_test = test_df['visual_appeal'].median()
test_df.loc[outlier_indices_va_test, 'visual_appeal'] = median_va_test

In [36]:
test_df.skew().sort_values(ascending = False)

headline_length           0.898307
headline_word_count       0.723191
time_of_day               0.378143
visual_appeal             0.242444
gender                    0.213702
seasonality               0.108320
day_of_week               0.083362
body_readability_score    0.022936
headline_sentiment        0.020468
body_keyword_density      0.005142
position_on_page          0.003744
body_sentiment           -0.011812
ad_frequency             -0.024512
ad_format                -0.029196
id                       -0.047338
body_text_length         -0.060726
market_saturation        -0.083123
age_group                -0.095668
headline_power_words     -0.096255
headline_numbers         -0.132487
targeting_score          -0.139761
headline_question        -0.152670
cta_strength             -0.162951
contextual_relevance     -0.164801
location                 -0.166878
brand_familiarity        -0.387485
device_type              -0.554649
body_word_count          -0.593898
dtype: float64

In [44]:
df_normalized_test

,visual_appeal,targeting_score,cta_strength,ad_format,contextual_relevance
0,0.298903,0.462756,0.244651,0.0,1.0
1,0.447603,0.649561,0.733952,0.5,0.0
2,0.064713,0.366726,0.632412,0.5,0.0
3,0.286048,0.598104,0.387762,0.5,0.0
4,0.641822,0.231378,0.244651,1.0,1.0
...,...,...,...,...,...
995,0.552503,0.366726,0.489301,1.0,1.0
996,0.489131,0.462756,0.489301,1.0,1.0
997,0.670411,0.366726,0.775524,0.5,0.0
998,0.624127,0.231378,0.568061,0.0,0.0


In [37]:
test = test_df[["visual_appeal", "targeting_score", "cta_strength", "ad_format", "contextual_relevance"]]

In [38]:
minmax1 = MinMaxScaler()
df_normalized_test = minmax1.fit_transform(test)
df_normalized_test = pd.DataFrame(df_normalized_test, columns=test.columns)

In [39]:
best_model.predict(df_normalized_test)

array([0.02603025, 0.0454996 , 0.01152976, 0.02542891, 0.06204843,
       0.12249166, 0.12948957, 0.08833439, 0.02159478, 0.03547318,
       0.03020893, 0.09300792, 0.01761612, 0.19204142, 0.08364619,
       0.06008508, 0.07372742, 0.15412818, 0.03671789, 0.0309963 ,
       0.24274984, 0.02261756, 0.04225646, 0.23044336, 0.02698986,
       0.03339351, 0.03832405, 0.06039136, 0.04141289, 0.06117782,
       0.02022216, 0.26387542, 0.03877406, 0.08811699, 0.04672071,
       0.02089376, 0.02758314, 0.0157082 , 0.02669899, 0.06852752,
       0.03896437, 0.05482651, 0.05123238, 0.17147526, 0.02873046,
       0.03946174, 0.03640993, 0.0309963 , 0.07054984, 0.05165069,
       0.04068633, 0.16797952, 0.02348098, 0.0352796 , 0.07114093,
       0.02875386, 0.03734714, 0.20243562, 0.02339096, 0.06541579,
       0.22506566, 0.04677013, 0.04550711, 0.03055824, 0.0754223 ,
       0.06401653, 0.03169423, 0.21845618, 0.10414196, 0.04080294,
       0.02832639, 0.02549261, 0.02299773, 0.04685936, 0.11344

In [40]:
test_df['CTR'] = best_model.predict(df_normalized_test)

In [41]:
test_df['id'] = test_df['id'].astype(int)

In [42]:
submit_df = test_df[['id', 'CTR']]

In [43]:
submit_df.to_csv("submit19.csv", index=False)

In [ ]:
m = pd.read_csv("sample_submission3.csv")
n = pd.read_csv("submit4.csv")

In [ ]:
m.dtypes

In [ ]:
mean_value